<a href="https://colab.research.google.com/github/Pager07/Webtech/blob/master/SVDHomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up

In [1]:
import pandas as pd 
import numpy as np
from scipy.sparse.linalg import svds

In [2]:
# from google.colab import drive 
# drive.mount('/content/gdrive/', force_remount = True)

#Loading Book and Rating Table



In [3]:
root_dir = '/Users/sandeep/Desktop/Homework2019/WebtechHomework/Webtech/flaskblog/BookData'
ratings_df = pd.read_csv(root_dir + '/ratev3_table.csv', sep=',', error_bad_lines=False,encoding="latin-1")
books_df = pd.read_csv(root_dir + '/books_table.csv',sep=',', error_bad_lines=False,encoding="latin-1")


In [4]:
ratings_df

,userID,ISBN,bookRating
0,2299,0805057706,10
1,2403,0345342968,9
2,2413,0679450408,6
3,2419,0679767800,7
4,2507,0385424736,9
...,...,...,...
3878,2146,0316769487,10
3879,2188,0870114573,7
3880,2227,051513306X,7
3881,2253,0805073337,9


In [5]:
books_df.head()

,ISBN,book_title,genre
0,0395366283,Curious George Goes to the Aquarium,Children's Fiction|General
1,0070434247,China: Its History and Culture,Asia - Civilization|Asian Studies - East Asia ...
2,2253141453,De l'amour et autres dÃ?ÃÂ©mons,Fiction
3,0425171299,Charmed,Romance fiction
4,0440508568,What Einstein Didn't Know : Scientific Answers...,Science|Essays


#Setting up Rating Data
 - Formate rating matrix 
    - user x movies 
    - use df.piviot ffunction
    - Fill cells with na with 0
- De-mean the data
    - nomrmalize by each users mean
        - find mean vector: user_mean * 1 
        - Subtract it from coloum
    - Convert it form a dataframe to numpy array

In [6]:
r_df = ratings_df.pivot(index='userID',columns='ISBN',values='bookRating').fillna(0)

In [7]:
r_df.head()

ISBN,0002211890,0002245663,0006161529,0006372570,000648381X,0007108389,000712614X,0020291604,002037500X,002086910X,...,843207036X,8437604184,843970206X,843970366X,847615562X,8478808175,848437386X,8830407445,8836807356,9728288115
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
R = r_df.as_matrix()
users_mean_rating = np.mean(R, axis = 1)
R_demeaned = R - users_mean_rating.reshape(-1,1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


#Singular Value Decomposition SVD
- Follow the guide
- Y = U x sigma x Vt
    - Kinda like X x Theta'
    - simga is a diagonal matrix like,identity matrix but not really
        - Just scales things up

- Y = user X movie

In [9]:
U,sigma,Vt = svds(R_demeaned, k=50)
sigma = np.diag(sigma)
Y = np.dot(np.dot(U,sigma),Vt) + users_mean_rating.reshape(-1,1)

In [10]:
Y_df = pd.DataFrame(Y,columns = r_df.columns)

In [11]:
Y_df.head()

ISBN,0002211890,0002245663,0006161529,0006372570,000648381X,0007108389,000712614X,0020291604,002037500X,002086910X,...,843207036X,8437604184,843970206X,843970366X,847615562X,8478808175,848437386X,8830407445,8836807356,9728288115
0,-0.005572,-0.004746,-0.005572,-0.004161,-0.005020,-0.004161,-0.005850,-0.004161,0.021200,-0.005066,...,-0.005020,0.204600,-0.005086,-0.004320,-0.004514,-0.004161,-0.004453,-0.005020,-0.005020,-0.005020
1,0.004536,0.004515,0.004536,0.004501,0.004522,0.004501,0.004543,0.004501,0.004406,0.003981,...,0.004522,0.000307,0.004524,0.004505,0.004510,0.004501,0.004508,0.004522,0.004522,0.004522
2,0.007283,0.007249,0.007283,0.007225,0.007261,0.007225,0.007294,0.007225,0.007067,0.006357,...,0.007261,0.000402,0.007263,0.007232,0.007240,0.007225,0.007237,0.007261,0.007261,0.007261
3,0.004573,0.004551,0.004573,0.004535,0.004558,0.004535,0.004580,0.004535,0.004431,0.003962,...,0.004558,0.000182,0.004560,0.004539,0.004544,0.004535,0.004543,0.004558,0.004558,0.004558
4,0.007335,0.007298,0.007335,0.007272,0.007311,0.007272,0.007347,0.007272,0.007101,0.006330,...,0.007311,0.000232,0.007313,0.007279,0.007288,0.007272,0.007285,0.007311,0.007311,0.007311


#Movie Reccomendation
 - Build a function that take userID and return you the sorted_user_prediction
    - Find which row the user is in?
         - I have made the dataset such that userID == row_index


In [12]:
def get_sorted_user_prediction(userId):
  user_row_number = userId 
  sorted_user_pred = Y_df.iloc[user_row_number].sort_values(ascending = False)
  return sorted_user_pred


In [13]:
get_sorted_user_prediction(4)

ISBN
0805073337    0.009630
0671721135    0.008907
0679404864    0.008879
0689859368    0.008860
156619024X    0.008478
                ...   
055358183X   -0.006082
0515133620   -0.006477
0812579569   -0.006552
0553801945   -0.008420
0670272663   -0.012694
Name: 4, Length: 1303, dtype: float64

In [14]:
def get_user_rated_books(books_dfx,ratings_dfx,userID):
  user_data = ratings_dfx[ratings_dfx.userID ==userID]
  user_full= user_data.merge(books_dfx,how='left',left_on = 'ISBN',right_on = 'ISBN').sort_values(['bookRating'],ascending=False)
  return user_full


In [15]:
def get_recommendation(books_df, ratings_df , userID):
  user_rated_books = get_user_rated_books(books_df,ratings_df,userID)
  books_not_rated_by_user = books_df[~books_df['ISBN'].isin(user_rated_books['ISBN'])]
  
  sorted_books_prediction_score_for_user = get_sorted_user_prediction(userID);
  sorted_books_prediction_score_for_user= pd.DataFrame(sorted_books_prediction_score_for_user).reset_index()
  highest_scored_unseen_books=(books_not_rated_by_user.merge(sorted_books_prediction_score_for_user,how='left',
                                                            left_on = 'ISBN',right_on = 'ISBN'));

  highest_scored_unseen_books = highest_scored_unseen_books.rename(columns={userID:'Prediction'}).sort_values('Prediction',ascending = False).iloc[:10, :-1]

  return user_rated_books , highest_scored_unseen_books



In [16]:
user_rated_books, highest_scored_unseen_books = get_recommendation(books_df, ratings_df, 20)

In [17]:
user_rated_books.head()

,userID,ISBN,bookRating,book_title,genre
0,20,0446527165,5,Wish You Well,Traffic Accident Victims|Family Relationships|...


In [18]:
highest_scored_unseen_books.head()

,ISBN,book_title,genre
1965,044020254X,The Empty House,Fiction|General
1963,0525938362,What I Lived for,City and town life|City and town life--New Yor...
529,067102096X,Lethal Seduction,Contemporary Romance|Women's Fiction
1635,0345455622,Lady Killer/Secret Admirer (2 Books in One),Paranormal & Fantastic Romance|Historical Romance
1161,0394557948,"Corpse Had a Familiar Face: Covering Miami, Am...","Buchanan, Edna|Journalists|Journalists--Florid..."


#Return dataframes

- book dataframe: books_df (named as)
- ratings dataframe: ratings_df (named as)
    - The changes that happens
        - ratings_df -> r_df (pandas dataframe) -> R(numpy matrix)

In [19]:
def get_df():
    return books_df, ratings_df

#Testing Different functions

In [56]:
user_data = ratings_df[ratings_df.userID ==494];user_data

,userID,ISBN,bookRating
2650,494,037540273X,8
2651,494,037575315X,5
2652,494,0375757902,5
2653,494,0393049922,10
2654,494,0449206513,5
2655,494,0515110469,5
2656,494,0618154000,5
2657,494,0670272663,5
2658,494,0670874868,5
2659,494,0679600000,5


In [62]:
user_full= user_data.merge(books_df,how='left',left_on = 'ISBN',right_on = 'ISBN').sort_values(['bookRating'],ascending=False);user_full

,userID,ISBN,bookRating,book_title,genre
3,494,0393049922,10,The Annotated Wizard of Oz: A Centennial Edition,"Wizard|Gale, Dorothy|Oz (Imaginary place)|Oz (..."
0,494,037540273X,8,Beloved : Gift Edition,Fiction|General
10,494,0679600922,5,"Snopes : The Hamlet, The Town, The Mansion (Mo...",Snopes Family (fictitious Characters)|Yoknapat...
16,494,0940450089,5,Nathaniel Hawthorne : Collected Novels: Fansha...,"Historical fiction, American|Literature"
15,494,0838315291,5,Charlotte Bronte: A Psychological Study (Engli...,"Psychology|Novelists, English|Fiction|Authorsh..."
14,494,081420256X,5,French and Italian Notebooks (Centenary Editio...,Literary Criticism|American
13,494,0811480542,5,Viva Mexico!: The Story of Benito Juarez and C...,Juvenile Literature|Presidents|Biography|Zapot...
12,494,080840475X,5,Hawthorne's Tales (Masterworks of Literature),Literary Collections
11,494,067960166X,5,The Basic Writings of Sigmund Freud (Psychopat...,Psychoanalytical Psychology
9,494,0679600000,5,Basic Writings of Nietzsche (Modern Library),Philosophy


In [0]:
userIDs = ratings_df.userID.unique()
xx = []
for userID in userIDs:
  user_data = ratings_df[ratings_df.userID == userID]
  if user_data.shape[0] > 4:
    xx.append(userID)

In [79]:
books_not_rated_by_user = books_df[~books_df['ISBN'].isin(user_full['ISBN'])]
sorted_books_prediction_score_for_user = get_sorted_user_prediction(494);
sorted_books_prediction_score_for_user= pd.DataFrame(sorted_books_prediction_score_for_user).reset_index()

highest_scored_unseen_books=(books_not_rated_by_user.merge(sorted_books_prediction_score_for_user,how='left',
                                                           left_on = 'ISBN',right_on = 'ISBN'));

highest_scored_unseen_books.rename(columns={494:'Prediction'}).sort_values('Prediction',ascending = False).iloc[:10, :-1]


,ISBN,book_title,genre
1178,067973452X,Notes from Underground (Vintage Classics),World Literature|Fiction Subjects|Peoples & Cu...
1913,0394709306,The Complete Grimm's Fairy Tales (Pantheon Fai...,"Fantasy fiction, Chinese--Translations into En..."
335,030723116X,Early Bird on Sesame Street,Literary Collections
439,038070384X,Lullaby,Fiction|Mystery & Detective
1822,076111937X,Amazing Gracie,Pets|Dogs
1492,0440505941,The Weekend Novelist,Fiction|Technique
212,157954519X,Ain't Misbehavin: The Groundbreaking Program f...,Pets|Cats
1840,0688088686,Tell Me No Secrets,Fiction|General
440,002086910X,OUT OF THE SILENT PLANET,Fiction|Science Fiction
2188,0895945398,Kitty Libber : Cat Cartoons by Women,Cats|Cats--Caricatures and cartoons|Wit and hu...
